# New strategy: concatenante 2010-2016 DFs. 
## Run LR, Random Forest on resultign DF. 

In [1]:
#Load necessary packages
import pandas as pd
import os
import numpy as np
import copy

In [3]:
#Load data created in Library_Week1_Notebook2_DataCleanUp

#Load data
os.chdir("//Users/Olga/Documents/INSIGHT2019/Library data/AllPldData")
Files_in_folder = os.listdir()
File_names = ['_2014', '_2015', '_2016', '_2001', '_2002', '_2003', 
              '_2004','_2005', '_2006', '_2007', '_2008', '_2009', 
              '_2010', '_2011', '_2012', '_2013', '_2000', '_1992', 
              '_1993', '_1994', '_1995', '_1996', '_1997', '_1998', '_1999']
Files = []

for filename in os.listdir():
    if filename.endswith('csv'):
        Files.append(pd.read_csv(filename, encoding = 'latin-1', low_memory = False))
        
#create dictionary of libraries labeled by year
LibData_dict = {}
for i in range(0, len(File_names)):
    LibData_dict[File_names[i]] = Files[i]
    
for k, v in LibData_dict.items():
    print(k, v.shape, type(LibData_dict[k]))

ColumnsPresent = pd.DataFrame(index=range(0),columns=range(0))

for k, v in LibData_dict.items():
    dataframe_to_join = pd.DataFrame({k: list(v.columns)})
    ColumnsPresent = ColumnsPresent.join(dataframe_to_join, how = 'outer')

#Rearrange columns in dataframe. 
cols = ColumnsPresent.columns.tolist()
cols_rearr = cols[17:] + cols[0:17]
cols_rearr2 = cols_rearr[0:8] + cols_rearr[24:25] + cols_rearr[11:24] + cols_rearr[8:11]

ColumnsPresent_rearr = ColumnsPresent[cols_rearr2]

ColumnsPresent_rearr_str = ColumnsPresent_rearr.applymap(str)
ColumnsPresent_rearr_str = ColumnsPresent_rearr_str.reset_index(drop = True)

#create DF of unique column values
UniqueLabels = np.unique(ColumnsPresent_rearr_str.values)
LabelsBoolDF = pd.DataFrame(UniqueLabels)

#Create DF w boolean values for prensence of each unique value
#THIS IS THE END GOAL OF THIS DATA
for j in range(len(ColumnsPresent_rearr_str.columns)):
    lis = []
    for i in range(len(UniqueLabels)):
        lis.append(ColumnsPresent_rearr_str.iloc[:, j].isin([UniqueLabels[i]]).any())
    LabelsBoolDF[ColumnsPresent_rearr_str.columns[j]] = lis
    
## Sum all TRUE values. Value of 25 = all true
sum_list = []
for i in range(LabelsBoolDF.shape[0]):
    sum_list.append(sum(LabelsBoolDF.iloc[i, 1:26]))

#Label OK in Notes
LabelsBoolDF['TRUE SUM'] = sum_list
LabelsBoolDF['Notes'] = ""
LabelsBoolDF.loc[LabelsBoolDF[LabelsBoolDF['TRUE SUM']==25].index, 'Notes'] = "OK"

#rows in which not all TRUE (meaning this column/data does not exist for all year)
MissingData = LabelsBoolDF[LabelsBoolDF['TRUE SUM']!=25].index.tolist()

#Columns in which data is unavailable for all years.
LabelsBoolDF.iloc[1,27] = "OK - Mailing address. Not necessary for analysis."
LabelsBoolDF.iloc[2,27] = "OK - Address type. Not necessary for analysis."
LabelsBoolDF.iloc[3,27] = "ATTEND was called VISITS after 1998. Combine with VISITS."
LabelsBoolDF.iloc[4:7,27] = "AUDIO is divided into AUDIO_PH, AUDIO_DL starting 2010. Source: 2010 Documentation."
LabelsBoolDF.iloc[12,27] = "Capital revenue, only computed after 2003. No equivalent prior, it seems."
LabelsBoolDF.iloc[13,27] = "OK - Call Based Statistical Area. Not necessary for analysis"
LabelsBoolDF.iloc[14,27] = "OK - Congressional district code. Not necessary for analysis."
LabelsBoolDF.iloc[15,27] = "OK - Census block. Not necessary for analysis."
LabelsBoolDF.iloc[17,27] = "OK - Census track. Not necessary for analysis."
LabelsBoolDF.iloc[19,27] = "OK - CITY_M. Not necessary for analysis."
LabelsBoolDF.iloc[21,27] = "OK - county code. Not necessary for analysis."
LabelsBoolDF.iloc[22,27] = "County population. Incomplete. Probably OK. Just need to make sure the there is another better measure of population - POPU_LSA I think."
LabelsBoolDF.iloc[24,27] = "OK - FSCS Public Library Definition. Not necessary for analysis."
LabelsBoolDF.iloc[25,27] = "C_LEGBAS - Legal Basis Code. SAME as C_LEGBASE 1998 and ON. Combine the two rows."
LabelsBoolDF.iloc[26,27] = "C_LEGBASE - Legal Basis Code. SAME as C_LEGBAS 1997 and prior. Combine the two rows."
LabelsBoolDF.iloc[28,27] = "DATABASE - Total licensed databases."
LabelsBoolDF.iloc[29,27] = "DB_LOC - Local licensed databases"
LabelsBoolDF.iloc[30,27] = "DB_LO_OT - Local/other databases"
LabelsBoolDF.iloc[31:33,27] = "More database info"
LabelsBoolDF.iloc[33,27] = "DUPLI. Called HRS_OPEN 1998 ON. Combine with HRS_OPEN."
LabelsBoolDF.iloc[34,27] = "EBOOK. Starts 2003 (makes sense)"
LabelsBoolDF.iloc[35,27] = "EC_LO_OT. Local/other electronic collections. 2015, 2016 only."
LabelsBoolDF.iloc[36,27] = "EC_ST. State electronic collections. 2015, 2016 only."
LabelsBoolDF.iloc[37,27] = "ELACCEXP. Operating expendature for electronic access."
LabelsBoolDF.iloc[38,27] = "ELCONT. 2016 only [2016 doc: first collection]. Electronic content use. Total circulation."
LabelsBoolDF.iloc[39,27] = "ELECCOLL. Total electronic collections. 2015, 2016 only."
LabelsBoolDF.iloc[40,27] = "ELINFO. Retrival of electronic infomation. 2016 only. "
LabelsBoolDF.iloc[41,27] = "ELMATCIRC. Circulation of electronic material. 2013-2016."
LabelsBoolDF.iloc[42,27] = "ELMATEXP. 1995-2016. Circulation of electronic materials. This is a good one to use!"
LabelsBoolDF.iloc[43,27] = "ELMATS. Number of library materials in electronic format. 1995-2002"
LabelsBoolDF.iloc[44,27] = "ELSVCACC. Library access to electronic services. Yes, no. Unnecessary. 1995-2002."
LabelsBoolDF.iloc[45,27] = "OK - ENDDATE. Not necessary for analysis."
LabelsBoolDF.iloc[46,27] = "ERES_USR. 1999-2005. Electronic resource users per year. "
LabelsBoolDF.iloc[47,27] = "ESUBSCRP. 2003-2009. Electronic subscriptions."
LabelsBoolDF.iloc[48,27] = "FCAP_REV. Federal gov capital revenue. 2005 and on."
LabelsBoolDF.iloc[50:61,27] = "OK - FILL. Not necessary for analysis."
LabelsBoolDF.iloc[61,27] = "OK - county code. Not necessary for analysis."
LabelsBoolDF.iloc[62,27] = "OK - place code. Not necessary for analysis."
LabelsBoolDF.iloc[63,27] = "OK - state code. Not necessary for analysis."
LabelsBoolDF.iloc[65:133,27] = "OK - imputation flags. Not necessary for analysis."
LabelsBoolDF.iloc[133,27] = "OK - GAL. Not necessary for analysis. Geocode location."
LabelsBoolDF.iloc[134,27] = "OK - GALMS. Not necessary for analysis."
LabelsBoolDF.iloc[135,27] = "GEOCODE - Geographical code. Might be useful. Can add to old datasets that don't have it."
LabelsBoolDF.iloc[136,27] = "OK - ID. Not necessary."
LabelsBoolDF.iloc[137,27] = "GPTERMS. Internet computers used by general public. 1998-2016."
LabelsBoolDF.iloc[138,27] = "HRS_OPEN. Called DUPLI 1997 and prior. Combine with DUPLI."
LabelsBoolDF.iloc[139:181,27] = "OK - imputation flags. Not necessary for analysis."
LabelsBoolDF.iloc[181,27] = "OK - County code. Not necassary for analysis."
LabelsBoolDF.iloc[182,27] = "OK - code. Not necassary for analysis."
LabelsBoolDF.iloc[183,27] = "INETACC - Internet access."
LabelsBoolDF.iloc[184,27] = "OK - imputation flag"
LabelsBoolDF.iloc[185,27] = "Kid program attendance. Combine with KIDATTEN."
LabelsBoolDF.iloc[186,27] = "Kid program attendance. Combine with KIDATTEND."
LabelsBoolDF.iloc[188,27] = "KIDPRO - Total kids program attendance."
LabelsBoolDF.iloc[189,27] = "OK - LATITUDE. NOt necessary for analysis."
LabelsBoolDF.iloc[190,27] = "LCAP_REV. Local gov capital revenue."
LabelsBoolDF.iloc[193,27] = "LIBRARIAN. Combine with LIBRARIA."
LabelsBoolDF.iloc[194,27] = "LIBRARIA. Combine with LIBRARIAN."
LabelsBoolDF.iloc[197,27] = "Urban-centric locale code. 2008-2016. 2008 doc: started in 2008. Can add to older years if necessary."
LabelsBoolDF.iloc[199,27] = "OK - LOGITUDE. Not necessary for analysis."
LabelsBoolDF.iloc[200,27] = "OK - LSABOUND. Legal services boundary. Not necessary for analysis."
LabelsBoolDF.iloc[202,27] = "OK - MAT_CENT. Match centroid. Not necessary for analysis."
LabelsBoolDF.iloc[203,27] = "OK - MAT_TYPE. Match status code. Not necessary for analysis."
LabelsBoolDF.iloc[204,27] = "OK - MICROF. Metro, micropolitan. Not necessary for analysis."
LabelsBoolDF.iloc[205,27] = "OK - MSTATUS. Match status. Not necessary for analysis." 
LabelsBoolDF.iloc[207,27] = "OCAP_REV. Other capital revenue."
LabelsBoolDF.iloc[209,27] = "OTHMATEX. Operating expenditures for all other library materials."
LabelsBoolDF.iloc[213,27] = "PHYSCIR. First collected in 2016."
LabelsBoolDF.iloc[214,27] = "PITUSR. Public use of internet."
LabelsBoolDF.iloc[215,27] = "POPU. Combine with POPU_LSA."
LabelsBoolDF.iloc[216,27] = "POPU_LSA. Population of legal service area. Combine with POPU."
LabelsBoolDF.iloc[217,27] = "POPU_UND. Combine with POPU_UNDUP."
LabelsBoolDF.iloc[218,27] = "POPU_UNDUP. Combine with POPU_UND."
LabelsBoolDF.iloc[219,27] = "OK - POSTMS. Postal match status. Unnecessary for analysis."
LabelsBoolDF.iloc[220,27] = "PRIMATEXP - Print material expenditures."
LabelsBoolDF.iloc[221,27] = "PUB_FIPS - OK. State code. Not necessary for analysis."
LabelsBoolDF.iloc[222,27] = "REAPLOCALE - urbancentric location code. Only 2016. Anything similar used before?"
LabelsBoolDF.iloc[223,27] = "REFERENC. Combine with REFERENCE."
LabelsBoolDF.iloc[224,27] = "REFERENCE. Combine with REFERENC."
LabelsBoolDF.iloc[225,27] = "REGBOR. Registered users. Anything similar prior to 2006?"
LabelsBoolDF.iloc[228,27] = "SCAP_REV. State revenue."
LabelsBoolDF.iloc[229,27] = "OK - SCORE. Address match. Not necessary for analysis."
LabelsBoolDF.iloc[231,27] = "STAFFEXP - Total staff exp. Salary and benefits. Combine with TOTEXP."
LabelsBoolDF.iloc[232,27] = "OK - STARTDAT. Reporting period start date. Not necessary for analysis."
LabelsBoolDF.iloc[233,27] = "OK - STATADDR. Address change code. Not necessary for analysis."
LabelsBoolDF.iloc[234,27] = "OK - STATNAME. Name change code. Not necessary for analysis."
LabelsBoolDF.iloc[235,27] = "OK - STATSTRU. Structure change code. Not necessary for analysis."
LabelsBoolDF.iloc[236,27] = "OK - STFTERMS. Internal terminals used by staff only. Not necessary for analysis."
LabelsBoolDF.iloc[238,27] = "SUBSCRIP. Combine with SUBSCRIPT."
LabelsBoolDF.iloc[239,27] = "SUBSCRIPT. Combine with SUBSCRIP."
LabelsBoolDF.iloc[240,27] = "TOTATTEN. Total attendance at library programs."
LabelsBoolDF.iloc[242,27] = "TOTCOLL. Total collection use. Only 2016."
LabelsBoolDF.iloc[243,27] = "TOTEXP. TOtal staff expenditures. Salary and benefits. Combine with STAFFEXP."
LabelsBoolDF.iloc[244,27] = "TOTEXPCO. Total expenditure of lib collection. Combine with TOTEXPCOL."
LabelsBoolDF.iloc[245,27] = "TOTEXPCOL. Total expenditure of lib collection. Combine with TOTEXPCO."
LabelsBoolDF.iloc[247,27] = "TOTOPEXP. Total operating expenditures. Combine with TOTOPEXP1."
LabelsBoolDF.iloc[248,27] = "TOTOPEXP1. Total operating expenditures. Combine with TOTOPEXP."
LabelsBoolDF.iloc[249,27] = "TOTPEMP - Total paid FTE employees. Combine with TOTSTAFF."
LabelsBoolDF.iloc[250,27] = "TOTPRO - Total library programs. Can this be combined with another label?"
LabelsBoolDF.iloc[251,27] = "TOTSTAFF. Total paid FTE employees. Combine with TOTPEMP."
LabelsBoolDF.iloc[252:255,27] = "VIDEO was later split into VIDEO_DL and VIDEO_PH."
LabelsBoolDF.iloc[255,27] = "VISITS. Combine with ATTEND."
LabelsBoolDF.iloc[256,27] = "OK - WEB_ADDR. Not necessary for data."
LabelsBoolDF.iloc[257,27] = "WIFISESS. Any similar metric prior?"
LabelsBoolDF.iloc[258,27] = "YAATTEN. Young adult program attendance."
LabelsBoolDF.iloc[259,27] = "YAPRO. Total young adult programs."
LabelsBoolDF.iloc[260:262,27] = "OK - Submission year.  YR and YR_SUB are the same. Not necessary for analysis."
LabelsBoolDF.iloc[260:262,27] = "OK - Submission year.  YR and YR_SUB are the same. Not necessary for analysis."
LabelsBoolDF.iloc[262,27] = "ZIP. Combine with ZIP1."
LabelsBoolDF.iloc[263,27] = "ZIP1. Combine with ZIP."
LabelsBoolDF.iloc[264:268,27] = "OK - More zip information. Not necessary for analysis."
LabelsBoolDF.iloc[268,27] = "Note that lack of nan."

#Column names to keep
#LabelsBoolDF.loc[[0, 3:13, 15], 0].tolist()
Cols_to_keep_idx = [0]
Cols_to_keep_idx.extend(range(3,13))
Cols_to_keep_idx.extend((16, 18, 20, 25, 26))
Cols_to_keep_idx.extend(range(33, 49))
Cols_to_keep_idx.extend((64, 135, 137, 138, 183))
Cols_to_keep_idx.extend(range(185, 189))
Cols_to_keep_idx.extend(range(191, 199))
Cols_to_keep_idx.append(201)
Cols_to_keep_idx.extend(range(206, 219))
Cols_to_keep_idx.append(220)
Cols_to_keep_idx.extend(range(222, 229))
Cols_to_keep_idx.extend((230, 231, 237))
Cols_to_keep_idx.extend(range(237, 256))
Cols_to_keep_idx.extend(range(257, 260))
Cols_to_keep_idx.extend((262, 263))

Cols_to_keep_in_lib_dfs = LabelsBoolDF.iloc[Cols_to_keep_idx, 0].tolist()

#Make dictionaries of smaller dataframe with Cols_to_keep_in_lib_dfs ONLY
LibData_dict_reduced = dict()

for k, v in LibData_dict.items():
    LibData_dict_reduced[k] = pd.DataFrame()
    for i in Cols_to_keep_in_lib_dfs:
        if i in v:
            TempList = list(v[i])
            LibData_dict_reduced[k][i] = TempList

#Did above loop do what I thought it did?
print("Original dfs:")
for k, v in LibData_dict.items():
    print(k, v.shape)

print("\n", "Reduced dfs:")  
for k, v in LibData_dict_reduced.items():    
    print(k, v.shape)

#Combine col names that mean same thing in different years into single 
Names_to_combine = ["ATTEND", "VISITS",  "C_LEGBAS", "C_LEGBASE", "DUPLI", "HRS_OPEN", "KIDATTEN", "KIDATTEND", 
                    "LIBRARIAN", "LIBRARIA", "POPU", "POPU_LSA", "POPU_UND", "POPU_UNDUP", "REFERENC", "REFERENCE", 
                    "STAFFEXP", "TOTEXP", "SUBSCRIP", "SUBSCRIPT", "TOTEXPCO", "TOTEXPCOL", 
                    "TOTOPEXP", "TOTOPEXP1", "TOTPEMP", "TOTSTAFF", "ZIP", "ZIP1"]

for i in range(0, len(Names_to_combine)-1, 2):
    for k, v in LibData_dict_reduced.items(): 
        if Names_to_combine[i] in v:
            name = Names_to_combine[i]
            newname = Names_to_combine[i] + "-" + Names_to_combine[i+1]
            v = v.rename(columns = {name: newname}, inplace = True)
        elif Names_to_combine[i+1] in v:
            name = Names_to_combine[i + 1]
            newname = Names_to_combine[i] + "-" + Names_to_combine[i+1]
            v = v.rename(columns = {name : newname}, inplace = True)

#Check that above code worked - names changes
for k, v in LibData_dict_reduced.items():
    print(v.columns)

for k, v in LibData_dict_reduced.items():    
    print(k, v.shape)

#Columns to add to DFs
"AUDIO is divided into AUDIO_PH, AUDIO_DL starting 2010. Source: 2010 Documentation."
"VIDEO was later split into VIDEO_DL and VIDEO_PH."

for k, v in LibData_dict_reduced.items(): 
    if "AUDIO_DL" in v:
        v["AUDIO"] = v['AUDIO_DL'] + v['AUDIO_PH']
    if "VIDEO_DL" in v:
        v["VIDEO"] = v['VIDEO_DL'] + v['VIDEO_PH']

#Check that # columns in some dataframes increased by 2 (expected)
for k, v in LibData_dict_reduced.items():    
    print(k, v.shape)
    
#Create DF w boolean values for prensence of each unique value in REDUCED dataframes
#THIS IS THE END GOAL OF THIS DATA

#Get all unique columns present in LibData_dict_reduced dataframes. 
ColumnsPresent_reduced = pd.DataFrame(index=range(0),columns=range(0))

for k, v in LibData_dict_reduced.items():
    dataframe_to_join_reduced = pd.DataFrame({k: list(v.columns)})
    ColumnsPresent_reduced = ColumnsPresent_reduced.join(dataframe_to_join_reduced, how = 'outer')

ColumnsPresent_reduced_str = ColumnsPresent_reduced.applymap(str)
ColumnsPresent_reduced_str = ColumnsPresent_reduced_str.reset_index(drop = True)
    
UniqueLabels_reduced = np.unique(ColumnsPresent_reduced_str.values)
LabelsBoolDF_reduced = pd.DataFrame(UniqueLabels_reduced)

#Create DF w boolean values for prensence of each unique value
#THIS IS THE END GOAL OF THIS DATA
for j in range(len(ColumnsPresent_reduced_str.columns)):
    lis = []
    for i in range(len(UniqueLabels_reduced)):
        lis.append(ColumnsPresent_reduced_str.iloc[:, j].isin([UniqueLabels_reduced[i]]).any())
    LabelsBoolDF_reduced[ColumnsPresent_reduced_str.columns[j]] = lis

## Sum all TRUE values. Value of 25 = all true
sum_list_reduced = []
for i in range(LabelsBoolDF_reduced.shape[0]):
    sum_list_reduced.append(sum(LabelsBoolDF_reduced.iloc[i, 1:26]))

#Label OK in Notes
LabelsBoolDF_reduced['TRUE SUM'] = sum_list_reduced
LabelsBoolDF_reduced['Notes'] = ""
LabelsBoolDF_reduced.loc[LabelsBoolDF_reduced[LabelsBoolDF_reduced['TRUE SUM']==25].index, 'Notes'] = "OK"

All_Cols_in_Reduced_dfs = list(LabelsBoolDF_reduced[0])
All_Cols_in_Reduced_dfs

for k, v in LibData_dict_reduced.items():    
    print(k, v.shape)
    
#Delete columns deemed NOT useful for downstream analysis
NonUsefulColstoDelete = ['ENDDATE', 'LIBID', 'PHONE', 'POPU_UND-POPU_UNDUP', 'RSTATUS', 'TOTCOLL', 'ZIP-ZIP1']

LibData_dict_reduced_dropuseless = copy.deepcopy(LibData_dict_reduced)

for k, v in LibData_dict_reduced_dropuseless.items(): 
    for i in range(len(NonUsefulColstoDelete)):
        if NonUsefulColstoDelete[i] in v:
            v.drop([NonUsefulColstoDelete[i]], axis = 1, inplace = True)
            
#Check quickly whether columns were dropped
print("Reduced dfs:")
for k, v in LibData_dict_reduced.items():    
    print(k, v.shape)

print("\n", "Reduced and dropped useless dfs:")
for k, v in LibData_dict_reduced_dropuseless.items():    
    print(k, v.shape)

#Get rid of negative values (replace w NaN)
#For all DFs, replace (-1, -3, -9) values with NaN. 
LibData_dict_reduced_dropuseless_NegRepWNaN = copy.deepcopy(LibData_dict_reduced_dropuseless)

for k, v in LibData_dict_reduced_dropuseless_NegRepWNaN.items():
    v[(v == -1) | (v == -3) | (v == -9)] = np.nan
    
#See if this worked (looks like it did)
print(sum(LibData_dict_reduced_dropuseless['_2014']['AUDIO_DL'] < 0))
print(np.where((LibData_dict_reduced_dropuseless['_2014']['AUDIO_DL'] < 0) == True))

print(sum(LibData_dict_reduced_dropuseless_NegRepWNaN['_2014']['AUDIO_DL'].isnull()))
print(np.where(LibData_dict_reduced_dropuseless_NegRepWNaN['_2014']['AUDIO_DL'].isnull()))

_2014 (9305, 159) <class 'pandas.core.frame.DataFrame'>
_2015 (9251, 159) <class 'pandas.core.frame.DataFrame'>
_2016 (9252, 151) <class 'pandas.core.frame.DataFrame'>
_2001 (9133, 108) <class 'pandas.core.frame.DataFrame'>
_2002 (9141, 107) <class 'pandas.core.frame.DataFrame'>
_2003 (9214, 110) <class 'pandas.core.frame.DataFrame'>
_2004 (9210, 128) <class 'pandas.core.frame.DataFrame'>
_2005 (9201, 131) <class 'pandas.core.frame.DataFrame'>
_2006 (9211, 139) <class 'pandas.core.frame.DataFrame'>
_2007 (9217, 148) <class 'pandas.core.frame.DataFrame'>
_2008 (9284, 150) <class 'pandas.core.frame.DataFrame'>
_2009 (9299, 152) <class 'pandas.core.frame.DataFrame'>
_2010 (9308, 154) <class 'pandas.core.frame.DataFrame'>
_2011 (9315, 157) <class 'pandas.core.frame.DataFrame'>
_2012 (9305, 155) <class 'pandas.core.frame.DataFrame'>
_2013 (9309, 157) <class 'pandas.core.frame.DataFrame'>
_2000 (9078, 108) <class 'pandas.core.frame.DataFrame'>
_1992 (8944, 83) <class 'pandas.core.frame.DataF

_2014 (9305, 69)
_2015 (9251, 72)
_2016 (9252, 77)
_2001 (9133, 52)
_2002 (9141, 52)
_2003 (9214, 53)
_2004 (9210, 56)
_2005 (9201, 60)
_2006 (9211, 61)
_2007 (9217, 61)
_2008 (9284, 62)
_2009 (9299, 64)
_2010 (9308, 67)
_2011 (9315, 67)
_2012 (9305, 67)
_2013 (9309, 68)
_2000 (9078, 52)
_1992 (8944, 44)
_1993 (8929, 44)
_1994 (8920, 44)
_1995 (8981, 49)
_1996 (8946, 49)
_1997 (8968, 49)
_1998 (8966, 51)
_1999 (9048, 52)
Reduced dfs:
_2014 (9305, 69)
_2015 (9251, 72)
_2016 (9252, 77)
_2001 (9133, 52)
_2002 (9141, 52)
_2003 (9214, 53)
_2004 (9210, 56)
_2005 (9201, 60)
_2006 (9211, 61)
_2007 (9217, 61)
_2008 (9284, 62)
_2009 (9299, 64)
_2010 (9308, 67)
_2011 (9315, 67)
_2012 (9305, 67)
_2013 (9309, 68)
_2000 (9078, 52)
_1992 (8944, 44)
_1993 (8929, 44)
_1994 (8920, 44)
_1995 (8981, 49)
_1996 (8946, 49)
_1997 (8968, 49)
_1998 (8966, 51)
_1999 (9048, 52)

 Reduced and dropped useless dfs:
_2014 (9305, 63)
_2015 (9251, 66)
_2016 (9252, 70)
_2001 (9133, 47)
_2002 (9141, 47)
_2003 (9214, 48)


In [4]:
for k, v in LibData_dict_reduced_dropuseless_NegRepWNaN.items():
    print(k, v.shape)

_2014 (9305, 63)
_2015 (9251, 66)
_2016 (9252, 70)
_2001 (9133, 47)
_2002 (9141, 47)
_2003 (9214, 48)
_2004 (9210, 50)
_2005 (9201, 54)
_2006 (9211, 55)
_2007 (9217, 55)
_2008 (9284, 56)
_2009 (9299, 58)
_2010 (9308, 61)
_2011 (9315, 61)
_2012 (9305, 61)
_2013 (9309, 62)
_2000 (9078, 47)
_1992 (8944, 39)
_1993 (8929, 39)
_1994 (8920, 39)
_1995 (8981, 44)
_1996 (8946, 44)
_1997 (8968, 44)
_1998 (8966, 46)
_1999 (9048, 47)


In [5]:
#Add usage column to each library
for k, v in LibData_dict_reduced_dropuseless_NegRepWNaN.items():
    v['Usage'] = v['ATTEND-VISITS']/v['POPU-POPU_LSA']
    
#Check that column was added
for k, v in LibData_dict_reduced_dropuseless_NegRepWNaN.items():
    print(k, v.shape)

_2014 (9305, 64)
_2015 (9251, 67)
_2016 (9252, 71)
_2001 (9133, 48)
_2002 (9141, 48)
_2003 (9214, 49)
_2004 (9210, 51)
_2005 (9201, 55)
_2006 (9211, 56)
_2007 (9217, 56)
_2008 (9284, 57)
_2009 (9299, 59)
_2010 (9308, 62)
_2011 (9315, 62)
_2012 (9305, 62)
_2013 (9309, 63)
_2000 (9078, 48)
_1992 (8944, 40)
_1993 (8929, 40)
_1994 (8920, 40)
_1995 (8981, 45)
_1996 (8946, 45)
_1997 (8968, 45)
_1998 (8966, 47)
_1999 (9048, 48)


In [14]:
#Remove all rows (libraries) for which Usage data missing for any window!
LibwUsage = copy.deepcopy(LibData_dict_reduced_dropuseless_NegRepWNaN)
for k, v in LibwUsage.items():
    NullInd = v[v.Usage.isnull()].index
    print(k, v.shape)
    v = v.drop(NullInd, axis = 0)
    print(k, v.shape)

_2014 (9305, 64)
_2014 (9259, 64)
_2015 (9251, 67)
_2015 (9231, 67)
_2016 (9252, 71)
_2016 (9234, 71)
_2001 (9133, 48)
_2001 (9131, 48)
_2002 (9141, 48)
_2002 (9138, 48)
_2003 (9214, 49)
_2003 (9214, 49)
_2004 (9210, 51)
_2004 (9207, 51)
_2005 (9201, 55)
_2005 (9198, 55)
_2006 (9211, 56)
_2006 (9208, 56)
_2007 (9217, 56)
_2007 (9214, 56)
_2008 (9284, 57)
_2008 (9257, 57)
_2009 (9299, 59)
_2009 (9256, 59)
_2010 (9308, 62)
_2010 (9273, 62)
_2011 (9315, 62)
_2011 (9275, 62)
_2012 (9305, 62)
_2012 (9265, 62)
_2013 (9309, 63)
_2013 (9263, 63)
_2000 (9078, 48)
_2000 (9078, 48)
_1992 (8944, 40)
_1992 (8944, 40)
_1993 (8929, 40)
_1993 (8929, 40)
_1994 (8920, 40)
_1994 (8920, 40)
_1995 (8981, 45)
_1995 (8981, 45)
_1996 (8946, 45)
_1996 (8946, 45)
_1997 (8968, 45)
_1997 (8968, 45)
_1998 (8966, 47)
_1998 (8966, 47)
_1999 (9048, 48)
_1999 (9048, 48)


In [105]:
#Create DF w boolean values for prensence of each unique value in REDUCED dataframes
#THIS IS THE END GOAL OF THIS DATA

#Get all unique columns present in LibData_dict_reduced dataframes. 
ColumnsPresent_reduced = pd.DataFrame(index=range(0),columns=range(0))

for k, v in LibwUsage .items():
    dataframe_to_join_reduced = pd.DataFrame({k: list(v.columns)})
    ColumnsPresent_reduced = ColumnsPresent_reduced.join(dataframe_to_join_reduced, how = 'outer')
    
UniqueLabels_reduced = np.unique(ColumnsPresent_reduced_str.values)
LabelsBoolDF_reduced = pd.DataFrame(UniqueLabels_reduced)

#Create DF w boolean values for prensence of each unique value
#THIS IS THE END GOAL OF THIS DATA
for j in range(len(ColumnsPresent_reduced_str.columns)):
    lis = []
    for i in range(len(UniqueLabels_reduced)):
        lis.append(ColumnsPresent_reduced_str.iloc[:, j].isin([UniqueLabels_reduced[i]]).any())
    LabelsBoolDF_reduced[ColumnsPresent_reduced_str.columns[j]] = lis
    
#Reorder columns
colnames = LabelsBoolDF_reduced.columns.tolist()
col_order = colnames[0:1] + colnames[18:26] + colnames[17:18] + colnames[4:17] + colnames[1:4]
LabelsBoolDF_reduced = LabelsBoolDF_reduced[col_order]


In [106]:
LabelsBoolDF_reduced

,0,_1992,_1993,_1994,_1995,_1996,_1997,_1998,_1999,_2000,...,_2007,_2008,_2009,_2010,_2011,_2012,_2013,_2014,_2015,_2016
0,ADDRESS,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,ATTEND-VISITS,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,AUDIO,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,AUDIO_DL,False,False,False,False,False,False,False,False,False,...,False,False,False,True,True,True,True,True,True,True
4,AUDIO_PH,False,False,False,False,False,False,False,False,False,...,False,False,False,True,True,True,True,True,True,True
5,BENEFIT,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
6,BKMOB,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
7,BKVOL,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
8,BRANLIB,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
9,CAPITAL,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [107]:
## Sum all TRUE values. Value of 25 = all true
sum_list_reduced_all = []
for i in range(LabelsBoolDF_reduced.shape[0]):
    sum_list_reduced_all.append(sum(LabelsBoolDF_reduced.iloc[i, 1:26]))
    
sum_list_reduced_ten = []
for i in range(LabelsBoolDF_reduced.shape[0]):
    sum_list_reduced_ten.append(sum(LabelsBoolDF_reduced.iloc[i, 15:26]))
    
sum_list_reduced_six = []
for i in range(LabelsBoolDF_reduced.shape[0]):
    sum_list_reduced_six.append(sum(LabelsBoolDF_reduced.iloc[i, 19:26]))

#Label OK in Notes
LabelsBoolDF_reduced['All_Years_Present'] = sum_list_reduced_all
LabelsBoolDF_reduced['2006_2016_Present'] = sum_list_reduced_ten
LabelsBoolDF_reduced['2010_2016_Present'] = sum_list_reduced_six

In [140]:
#Based on this, I will use 2010-2016 DF. This has the most values.
LabelsBoolDF_reduced.loc[29: ,:]

,Col,_1992,_1993,_1994,_1995,_1996,_1997,_1998,_1999,_2000,...,_2010,_2011,_2012,_2013,_2014,_2015,_2016,All_Years_Present,2006_2016_Present,2010_2016_Present
29,ESUBSCRP,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,7,4,0
30,FCAP_REV,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,12,11,7
31,FSCSKEY,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,25,11,7
32,GEOCODE,False,False,False,False,False,False,True,True,True,...,True,True,True,True,True,True,True,19,11,7
33,GPTERMS,False,False,False,False,False,False,True,True,True,...,True,True,True,True,True,True,True,19,11,7
34,INETACC,False,False,False,True,True,True,True,True,True,...,False,False,False,False,False,False,False,8,0,0
35,KIDATTEN-KIDATTEND,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,25,11,7
36,KIDCIRCL,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,25,11,7
37,KIDPRO,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,12,11,7
38,LIBID,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,25,11,7


In [163]:
#Make sure each df has all 83 columns. If column did not exist, add this column with NaN
Unique_col_names_list = list(LabelsBoolDF_reduced['Col'])

LibwUsage_2 = copy.deepcopy(LibwUsage)

for k, v in LibwUsage_2.items():
    for i in range(len(Unique_col_names_list)):
        if Unique_col_names_list[i] not in v:
            v[Unique_col_names_list[i]] = np.nan

for k, v in LibwUsage_2.items():
    print(k, v.shape)

_2014 (9305, 85)
_2015 (9251, 85)
_2016 (9252, 85)
_2001 (9133, 85)
_2002 (9141, 85)
_2003 (9214, 85)
_2004 (9210, 85)
_2005 (9201, 85)
_2006 (9211, 85)
_2007 (9217, 85)
_2008 (9284, 85)
_2009 (9299, 85)
_2010 (9308, 85)
_2011 (9315, 85)
_2012 (9305, 85)
_2013 (9309, 85)
_2000 (9078, 85)
_1992 (8944, 85)
_1993 (8929, 85)
_1994 (8920, 85)
_1995 (8981, 85)
_1996 (8946, 85)
_1997 (8968, 85)
_1998 (8966, 85)
_1999 (9048, 85)


In [165]:
Concat201116 = pd.concat([LibwUsage_2['_2010'], LibwUsage_2['_2011'], LibwUsage_2['_2012'], LibwUsage_2['_2013'], LibwUsage_2['_2014'], LibwUsage_2['_2015'], LibwUsage_2['_2016']], axis=0, sort = False, join = "outer")

In [166]:
Concat201116.shape

(65045, 85)

In [167]:
#rename first col
LabelsBoolDF_reduced.rename(columns = {LabelsBoolDF_reduced.columns[0] : "Col"}, inplace = True)

In [170]:
#Which features are not present in all 7 years?
Index = LabelsBoolDF_reduced[LabelsBoolDF_reduced['2010_2016_Present'] != 7].index

#Delete these features.
Cols_to_delete = LabelsBoolDF_reduced.loc[Index, 'Col']

#Delete from dataframe:
Cols_to_del_list = list(Cols_to_delete[:-1])
print("Columns to delete:", Cols_to_del_list)

#Delete columns
Concat201116_INCOMPLETECOLSDEL = Concat201116.drop(columns = Cols_to_del_list)



Columns to delete: ['EC_LO_OT', 'EC_ST', 'ELACCEXP', 'ELCONT', 'ELECCOLL', 'ELINFO', 'ELMATCIR', 'ELMATS', 'ELSVCACC', 'ERES_USR', 'ESUBSCRP', 'INETACC', 'PHYSCIR', 'REAPLOCALE', 'TOTCOLL', 'WIFISESS']


In [172]:
print(Concat201116.shape)
print(Concat201116_INCOMPLETECOLSDEL.shape)

(65045, 85)
(65045, 69)


In [207]:
#Nonuseful columns to delete from above:
NonUsefulColstoDelete = ['ENDDATE', 'LIBID', 'PHONE', 'POPU_UND-POPU_UNDUP', 'RSTATUS', 'ZIP-ZIP1', 'nan']
Concat201116_INCOMPLETECOLSDEL2 = Concat201116_INCOMPLETECOLSDEL.drop(columns = NonUsefulColstoDelete)
print(Concat201116_INCOMPLETECOLSDEL2.shape)

(65045, 62)


In [208]:
Concat201116_INCOMPLETECOLSDEL2.columns

Index(['ADDRESS', 'AUDIO_DL', 'AUDIO_PH', 'BENEFIT', 'BKMOB', 'BKVOL',
       'BRANLIB', 'CAPITAL', 'CAP_REV', 'CENTLIB', 'CITY', 'CNTY',
       'C_LEGBAS-C_LEGBASE', 'EBOOK', 'ELMATEXP', 'FCAP_REV', 'FSCSKEY',
       'GEOCODE', 'GPTERMS', 'DUPLI-HRS_OPEN', 'KIDATTEN-KIDATTEND',
       'KIDCIRCL', 'KIDPRO', 'LIBNAME', 'LIBRARIAN-LIBRARIA', 'LOANFM',
       'LOANTO', 'LOCALE', 'LOCGVT', 'MASTER', 'OBEREG', 'OCAP_REV', 'OTHINCM',
       'OTHMATEX', 'OTHOPEXP', 'OTHPAID', 'PITUSR', 'POPU-POPU_LSA',
       'PRMATEXP', 'REFERENC-REFERENCE', 'REGBOR', 'SALARIES', 'SCAP_REV',
       'STABR', 'STAFFEXP-TOTEXP', 'STGVT', 'SUBSCRIP-SUBSCRIPT', 'TOTATTEN',
       'TOTCIR', 'TOTEXPCO-TOTEXPCOL', 'TOTINCM', 'TOTOPEXP-TOTOPEXP1',
       'TOTPRO', 'TOTPEMP-TOTSTAFF', 'VIDEO_DL', 'VIDEO_PH', 'ATTEND-VISITS',
       'YAATTEN', 'YAPRO', 'AUDIO', 'VIDEO', 'Usage'],
      dtype='object')

In [ ]:
#I SHOULD CHECK THIS MORE CAREFULLY ON FRI!!

#Columns to move to front, not use 
'ADDRESS', 'ATTEND-VISITS', 'CITY', 'CNTY', 'FSCSKEY', 'LIBNAME',   

#Columns to delete prior to analysis
'AUDIO_DL', 'AUDIO_PH'

#Columns to categorize -- DELETE FOR NOW, but for tomorrow may was to categorize
'GEOCODE', 'LOCALE', 'OBEREG', 'STABR', 'C_LEGBAS-C_LEGBASE'

In [241]:
#Delete 'AUDIO_DL', 'AUDIO_PH'
Concat201116_INCOMPLETECOLSDEL3 = Concat201116_INCOMPLETECOLSDEL2.drop(columns = ['AUDIO_DL', 'AUDIO_PH'])
#Put 'ADDRESS', 'ATTEND-VISITS', 'CITY', 'CNTY', 'FSCSKEY', 'LIBNAME' in dif DF
Location_Detail_DF = Concat201116_INCOMPLETECOLSDEL2[['ADDRESS', 'ATTEND-VISITS', 'CITY', 'CNTY', 'FSCSKEY', 'LIBNAME']]

#NOW delete from concatDF
Concat201116_INCOMPLETECOLSDEL4 = Concat201116_INCOMPLETECOLSDEL3.drop(columns = ['ADDRESS', 'ATTEND-VISITS', 'CITY', 'CNTY', 'FSCSKEY', 'LIBNAME'])

#NOW concat 4 values
Concat_w_dummies = pd.get_dummies(Concat201116_INCOMPLETECOLSDEL4, columns=['GEOCODE', 'LOCALE', 'OBEREG', 'STABR', 'C_LEGBAS-C_LEGBASE'], prefix=['GEOCODE', 'LOCALE', 'OBEREG', 'STABR', 'C_LEGBAS-C_LEGBASE'])

In [242]:
print(Concat201116_INCOMPLETECOLSDEL2.shape, Concat201116_INCOMPLETECOLSDEL3.shape, Concat201116_INCOMPLETECOLSDEL4.shape, Concat_w_dummies.shape)

(65045, 62) (65045, 60) (65045, 54) (65045, 160)


In [297]:
#Get rid of columns with any NaN. Obviously, this is something to re-think later.

###RESET INDEX for BOTH
Concat_w_dummies = Concat_w_dummies.reset_index(drop = True)
Location_Detail_DF = Location_Detail_DF.reset_index(drop = True)

#25000 rows have NaN. Just DELETE These rows for now
print(Concat_w_dummies.shape)
print(len(pd.isnull(Concat_w_dummies).any(1).nonzero()[0]))

idx = pd.isnull(Concat_w_dummies).any(1).nonzero()[0]

#Delete_row_w_any_NaN
Concat_w_dummies_noNaN = Concat_w_dummies.drop(idx)
Location_Detail_DF_noNaN = Location_Detail_DF.drop(idx)

print(Concat_w_dummies_noNaN.shape, Location_Detail_DF_noNaN.shape)

(65045, 160)
25067
(39978, 160) (39978, 6)


In [317]:
import matplotlib.pylab as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [295]:
print(Concat_w_dummies_noNaN.dtypes[1:20])
print(Concat_w_dummies_noNaN.dtypes[20:40])
print(Concat_w_dummies_noNaN.dtypes[40:60])
print(Concat_w_dummies_noNaN.dtypes[60:100])

BKMOB                 float64
BKVOL                 float64
BRANLIB               float64
CAPITAL               float64
CAP_REV               float64
CENTLIB               float64
EBOOK                 float64
ELMATEXP              float64
FCAP_REV              float64
GPTERMS               float64
DUPLI-HRS_OPEN        float64
KIDATTEN-KIDATTEND    float64
KIDCIRCL              float64
KIDPRO                float64
LIBRARIAN-LIBRARIA    float64
LOANFM                float64
LOANTO                float64
LOCGVT                float64
MASTER                float64
dtype: object
OCAP_REV              float64
OTHINCM               float64
OTHMATEX              float64
OTHOPEXP               object
OTHPAID               float64
PITUSR                float64
POPU-POPU_LSA         float64
PRMATEXP              float64
REFERENC-REFERENCE    float64
REGBOR                float64
SALARIES               object
SCAP_REV              float64
STAFFEXP-TOTEXP        object
STGVT                 floa

In [314]:
#These 3 columns have non-float types and I don't know why!! 
#FOR NOW, I will delete columns. Definitely something to go back to 
Concat_w_dummies_noNaN_3othersdel = Concat_w_dummies_noNaN.drop(columns = ['OTHOPEXP', 'SALARIES', 'STAFFEXP-TOTEXP', 'BENEFIT'])


In [324]:
#Try to predict usage
X = Concat_w_dummies_noNaN_3othersdel.drop(columns = ['Usage']).astype(float)
y = Concat_w_dummies_noNaN_3othersdel['Usage'].astype(float)

normalized_X = preprocessing.normalize(X)

X_train, X_test, y_train, y_test = train_test_split(normalized_X, y, test_size=0.2, random_state=0)

model = LinearRegression()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))

Coefficients = list(model.coef_)

0.20491922024772957


In [357]:
Concat_w_dummies_noNaN_3othersdel['Usage']

1         2.599498
3         2.819901
4         4.281259
9        10.393033
11       51.981211
15        2.243814
19       33.072568
22       25.073156
24        7.351684
26       13.493803
27       11.795207
28       11.319181
29        6.129032
33       10.180542
36        6.114508
38        4.983333
40       23.138399
44       28.635722
45       16.622790
47       22.531588
50        5.128000
55        9.508300
56        2.994489
57        5.489393
58       13.094133
64        7.736272
75       26.598949
90        3.368558
95        3.862076
97        0.013581
           ...    
65007     1.938626
65008     1.627056
65009     4.337017
65010     4.038326
65011     1.590168
65013     1.503987
65016     4.819440
65017     5.592958
65018     3.393408
65021     2.081655
65022     3.573480
65023     5.073121
65024     7.038745
65025     4.723829
65026     8.982790
65027     2.423740
65028     5.737379
65029     7.512638
65030     4.455791
65031     5.014224
65032     4.022109
65033     8.

In [361]:
len(Coefficients)

155

In [363]:
#Save predictions (this was fast and messy)
Predictions = np.dot(normalized_X, Coefficients)
df1 = Location_Detail_DF_noNaN[['LIBNAME']]
df1['new'] = Predictions


//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [381]:
FlaskCSV2libs =  df1.iloc[0:9000, :].to_csv(path_or_buf = "/Users/Olga/Documents/INSIGHT2019/FLASK_Week2Demo/flaskweek2/FlaskCSV_2.csv")

In [373]:
df1[df1.LIBNAME == 'KUSKOKWIM CONSORTIUM LIBRARY'].index

Int64Index([3, 9311, 18626, 27931, 37240, 46545, 55796], dtype='int64')

In [380]:
df1.iloc[df1[df1.LIBNAME == 'KUSKOKWIM CONSORTIUM LIBRARY'].index[0], 1]

-4.775874989432279

In [383]:
df1

,LIBNAME,new
1,ANCHORAGE PUBLIC LIBRARY,-6.303555
3,KUSKOKWIM CONSORTIUM LIBRARY,-4.533930
4,BIG LAKE PUBLIC LIBRARY,-4.733666
9,CORDOVA PUBLIC LIBRARY,-4.775875
11,DELTA COMMUNITY LIBRARY,-1.082918
15,FAIRBANKS NORTH STAR BOROUGH PUBLIC LIBRARY,-6.228966
19,HAINES BOROUGH PUBLIC LIBRARY,-2.887354
22,HOMER PUBLIC LIBRARY,-3.865808
24,JUNEAU PUBLIC LIBRARIES,-5.804328
26,KENAI COMMUNITY LIBRARY,-4.866536


# After Friday Demo

In [463]:
#Above, I found that 4 columns - 'OTHOPEXP', 'SALARIES', 'STAFFEXP-TOTEXP', 'BENEFIT' - are objects, meaning some balues are non-numeric
#It looks like some are left blank.
print(Concat_w_dummies_noNaN[Concat_w_dummies_noNaN.OTHOPEXP == ' '].index)
print(Concat_w_dummies_noNaN[Concat_w_dummies_noNaN.SALARIES == ' '].index)
print(Concat_w_dummies_noNaN[Concat_w_dummies_noNaN['STAFFEXP-TOTEXP'] == ' '].index)
print(Concat_w_dummies_noNaN[Concat_w_dummies_noNaN.BENEFIT == ' '].index)

#Are all indices the same? YES. 
print(len(Concat_w_dummies_noNaN[Concat_w_dummies_noNaN.OTHOPEXP == ' '].index.intersection(Concat_w_dummies_noNaN[Concat_w_dummies_noNaN.SALARIES == ' '].index)))
print(len(Concat_w_dummies_noNaN[Concat_w_dummies_noNaN.OTHOPEXP == ' '].index.intersection(Concat_w_dummies_noNaN[Concat_w_dummies_noNaN['STAFFEXP-TOTEXP'] == ' '].index)))
print(len(Concat_w_dummies_noNaN[Concat_w_dummies_noNaN.OTHOPEXP == ' '].index.intersection(Concat_w_dummies_noNaN[Concat_w_dummies_noNaN.BENEFIT == ' '].index)))

IndexToDropSpaces = Concat_w_dummies_noNaN[Concat_w_dummies_noNaN.OTHOPEXP == ' '].index


Int64Index([37237, 37239, 37242, 37243, 37244, 37245, 37247, 37248, 37250,
            37251,
            ...
            46487, 46488, 46490, 46499, 46501, 46502, 46503, 46510, 46516,
            46517],
           dtype='int64', length=3309)
Int64Index([37237, 37239, 37242, 37243, 37244, 37245, 37247, 37248, 37250,
            37251,
            ...
            46487, 46488, 46490, 46499, 46501, 46502, 46503, 46510, 46516,
            46517],
           dtype='int64', length=3309)
Int64Index([37237, 37239, 37242, 37243, 37244, 37245, 37247, 37248, 37250,
            37251,
            ...
            46487, 46488, 46490, 46499, 46501, 46502, 46503, 46510, 46516,
            46517],
           dtype='int64', length=3309)
Int64Index([37237, 37239, 37242, 37243, 37244, 37245, 37247, 37248, 37250,
            37251,
            ...
            46487, 46488, 46490, 46499, 46501, 46502, 46503, 46510, 46516,
            46517],
           dtype='int64', length=3309)
3309
3309
3309


In [519]:
Concat_w_dummies_noNaN_noSpace = Concat_w_dummies_noNaN.drop(IndexToDropSpaces)
Location_Detail_DF_noNaN_noSpace = Location_Detail_DF_noNaN.drop(IndexToDropSpaces)
print(Concat_w_dummies_noNaN.shape, Location_Detail_DF_noNaN.shape)
print(Concat_w_dummies_noNaN_noSpace.shape, Location_Detail_DF_noNaN_noSpace.shape)

(39978, 160) (39978, 6)
(36669, 160) (36669, 6)


In [465]:
#Try to predict usage with Concat_w_dummies_noNaN_noSpace
X = Concat_w_dummies_noNaN_noSpace.drop(columns = ['Usage']).astype(float)
y = Concat_w_dummies_noNaN_noSpace['Usage'].astype(float)

normalized_X = preprocessing.normalize(X)

X_train, X_test, y_train, y_test = train_test_split(normalized_X, y, test_size=0.2, random_state=0)

model = LinearRegression()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))

Coefficients = list(model.coef_)

0.2248423250228356


In [468]:
#Save predictions (this was fast and messy)
Predictions_2 = np.dot(normalized_X, Coefficients)
print(Predictions_2)

[-3.55763691  2.3148254   2.41315668 ... -3.8116245  -4.69315023
 -3.20564899]


In [470]:
list(Predictions_2)

[-3.5576369094361753,
 2.314825400729553,
 2.4131566807686706,
 2.1444454822322916,
 9.150758800076765,
 -2.7869556075505377,
 3.571117519865425,
 1.289385288352617,
 -2.4953886595241515,
 -2.175438827588634,
 -0.7045070811082854,
 0.07025802397572989,
 4.501634530465172,
 3.9616403429954157,
 3.4997880591974933,
 -2.326154826109631,
 4.108561849903928,
 2.5049261191653756,
 0.8235510003651272,
 1.93603964957944,
 5.793128740192773,
 2.194343542545479,
 -2.4485941008479486,
 3.8609310195122233,
 3.8016467188548435,
 -1.6256772248103952,
 6.439430739396164,
 -13.983231081913942,
 -3.270148893484987,
 -38.530066291784216,
 -1.5214549170821052,
 -0.0948747042575631,
 0.13516595486048402,
 0.5379559600385162,
 -1.6701019982690035,
 -3.1017349578043683,
 2.693286074533112,
 -3.518258822840099,
 -5.429348109998428,
 -8.591360546010142,
 1.0010361167497088,
 -7.858566317075601,
 -3.7525483258450296,
 -2.017469073755038,
 -2.5543629189511248,
 -4.80781845694137,
 -7.4888730817020654,
 -3.29633

In [471]:
#Are predictions less 'negative' if I don't normalize dataframe first?

X = Concat_w_dummies_noNaN_noSpace.drop(columns = ['Usage']).astype(float)
y = Concat_w_dummies_noNaN_noSpace['Usage'].astype(float)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = LinearRegression()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))


Coefficients = list(model.coef_)
Predictions_3 = np.dot(normalized_X, Coefficients)
list(Predictions_3)

#YES, BUT predictions make even less sense. Number are tiny, v. close to 0. 

0.13594433999192257


[8.780693930155472e-08,
 9.34938742918678e-06,
 8.600014664375875e-06,
 1.1708502062915044e-05,
 2.2853178408591028e-05,
 2.7021643553349913e-07,
 7.4795837874431965e-06,
 6.171768520346195e-06,
 8.889320026302749e-07,
 9.710574192914633e-07,
 1.8668779381442267e-06,
 4.236115763717966e-06,
 1.3899605550047585e-05,
 1.7964136036783698e-05,
 1.4479930266720626e-05,
 1.350351191705595e-06,
 1.0039786480319186e-05,
 1.0185358578666227e-05,
 2.6301337819525225e-06,
 7.9995352407216e-06,
 1.595745719846881e-05,
 1.0923268468795035e-05,
 1.1016555917667206e-06,
 1.2504706178092828e-05,
 1.196217540099567e-05,
 2.018987725738699e-06,
 5.74845945517878e-06,
 -1.0582459555948288e-05,
 -2.5883930574883744e-06,
 -1.4240221127920586e-05,
 -3.938663840627736e-06,
 1.2428910580757027e-05,
 -3.314151112195932e-07,
 -1.8848492920901718e-06,
 -2.813589821679467e-06,
 8.045164210184397e-06,
 2.0419356884231423e-06,
 -8.202709962129745e-07,
 7.902135596800281e-06,
 8.834885094421074e-07,
 3.4025610904275

In [473]:
#Try random forest regressor
from sklearn.ensemble import RandomForestRegressor
rfmodel = RandomForestRegressor(n_estimators = 100, random_state = 42)

X = Concat_w_dummies_noNaN_noSpace.drop(columns = ['Usage']).astype(float)
y = Concat_w_dummies_noNaN_noSpace['Usage'].astype(float)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

rfmodel.fit(X_train, y_train);

predictions = rfmodel.predict(X_test)

errors = abs(predictions - y_test)

#df of predictions, labels and errors
df = pd.DataFrame(predictions, y_test)
df['errors'] = (list(errors))

#Works surprisingly well, am I missing something?

In [511]:
#Try random forest regressor WITHOUT population info
from sklearn.ensemble import RandomForestRegressor
rfmodel = RandomForestRegressor(n_estimators = 100, random_state = 42)

X = Concat_w_dummies_noNaN_noSpace.drop(columns = ['Usage', 'POPU_LSA']).astype(float)
y = Concat_w_dummies_noNaN_noSpace['Usage'].astype(float)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

rfmodel.fit(X_train, y_train);

predictions = rfmodel.predict(X_test)

errors = abs(predictions - y_test)

#df of predictions, labels and errors
df = pd.DataFrame(predictions, y_test)
df['errors'] = (list(errors))

#Also, works equally well. 

#Do I need to try this with just a single year?? Or take out handful (20%) of libraries from test frame to use as trial frame.

In [ ]:
#H2O random forest handles categorical variables

In [576]:
import random
print(Concat_w_dummies_noNaN.shape)
print(Concat_w_dummies_noNaN_noSpace.shape)
print(Location_Detail_DF_noNaN.shape)
print(Location_Detail_DF_noNaN_noSpace.shape)

Concat_w_dummies_noNaN_noSpace_ri = Concat_w_dummies_noNaN_noSpace.reset_index(drop = True)
Location_Detail_DF_noNaN_noSpace_ri = Location_Detail_DF_noNaN_noSpace.reset_index(drop = True)

UniqueFSCS = set(Location_Detail_DF_noNaN_noSpace_ri.loc[:, 'FSCSKEY'].tolist())
print(len(UniqueFSCS))

#Pick 20% of FSCSKEYS randomly
Numb_to_sel = int((.20*len(UniqueFSCS)))
_20PctFSCSList = random.sample(UniqueFSCS, Numb_to_sel)

#Get index of these FSCSKEYS from Location_Detail_DF_noNaN_noSpace_ri
Indexlist = list()
for FSCSkey in _20PctFSCSList:
    Indexfor_20PctFSCSList = Location_Detail_DF_noNaN_noSpace_ri[Location_Detail_DF_noNaN_noSpace_ri.FSCSKEY == FSCSkey].index
    Indexlist.append(list(Indexfor_20PctFSCSList))

flattened_Indexlist = [val for sublist in Indexlist for val in sublist]
print(len(flattened_Indexlist))

(39978, 160)
(36669, 160)
(39978, 6)
(36669, 6)
6303
7340


In [577]:
#RF with 
from sklearn.ensemble import RandomForestRegressor
rfmodel = RandomForestRegressor(n_estimators = 100, random_state = 42)

X = Concat_w_dummies_noNaN_noSpace_ri.drop(columns = ['Usage', 'POPU_LSA'])
y = Concat_w_dummies_noNaN_noSpace_ri['Usage']

X_train = X.drop(X.index[flattened_Indexlist])
y_train = y.drop(y.index[flattened_Indexlist])
X_test = X.loc[flattened_Indexlist]
y_test = y.loc[flattened_Indexlist]

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

rfmodel.fit(X_train, y_train);

predictions = rfmodel.predict(X_test)

errors = abs(predictions - y_test)

#df of predictions, labels and errors
df = pd.DataFrame(predictions, y_test)
df['errors'] = (list(errors))

In [578]:
df

,0,errors
Usage,,
11.930520,9.775477,2.155043
7.589655,9.750800,2.161146
8.293819,9.922175,1.628356
7.953599,9.600138,1.646538
8.646869,10.696896,2.050027
2.985737,4.743366,1.757628
3.078270,5.087329,2.009059
3.238518,4.527157,1.288639
4.104742,4.662114,0.557372
